In [8]:
import json
import requests
import sys
from datetime import datetime, time

sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')


from virasana.integracao.due import raspa_due

VIRASANA_URL = "http://localhost/virasana/"
VIRASANA_URL = "https://10.68.64.12/virasana/"

## Realizando consulta no MongoDB Gridfs do AJNA via API virasana

In [58]:
diaapesquisar = datetime.today()
diaapesquisar = datetime(2019, 11, 6)

In [59]:
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2019-11-06  00:00:00 2019-11-06 23:59:59


In [60]:
params = {'query': 
          { 'metadata.dataescaneamento': {'$gte': datainicial, '$lte': datafinal},
            'metadata.contentType': 'image/jpeg',
            'metadata.carga.manifesto.tipomanifesto': 'lce'
          },
          'projection': 
          {'metadata.numeroinformado': 1,
           'metadata.dataescaneamento': 1}
         }

r = requests.post(VIRASANA_URL + "grid_data", json=params, verify=False)
lista_lce = list(r.json())
print(len(lista_lce))

params['query']['metadata.carga.manifesto.tipomanifesto'] = None
r = requests.post(VIRASANA_URL + "grid_data", json=params, verify=False)
lista_null = list(r.json())
print(len(lista_null))

# print(r.url)
# print(r.text)

c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



12
1794


In [61]:
lista_lce

[{'_id': '5dc31beacdd9fc3e8b8828a0',
  'metadata': {'dataescaneamento': '2019-11-06 03:39:39',
   'numeroinformado': 'MNBU3980129'}},
 {'_id': '5dc31bebcdd9fc3e8b8829ad',
  'metadata': {'dataescaneamento': '2019-11-06 04:11:27',
   'numeroinformado': 'MRKU9909998'}},
 {'_id': '5dc31bf5cdd9fc3e8b882c1c',
  'metadata': {'dataescaneamento': '2019-11-06 04:42:16',
   'numeroinformado': 'TCLU2385790'}},
 {'_id': '5dc31c0dcdd9fc3e8b88330f',
  'metadata': {'dataescaneamento': '2019-11-06 07:03:30',
   'numeroinformado': 'MSKU2319040'}},
 {'_id': '5dc31c0dcdd9fc3e8b883373',
  'metadata': {'dataescaneamento': '2019-11-06 07:12:57',
   'numeroinformado': 'MNBU3841632'}},
 {'_id': '5dc31c0dcdd9fc3e8b88336f',
  'metadata': {'dataescaneamento': '2019-11-06 07:16:57',
   'numeroinformado': 'MNBU0401157'}},
 {'_id': '5dc31c2ecdd9fc3e8b883b55',
  'metadata': {'dataescaneamento': '2019-11-06 07:19:42',
   'numeroinformado': 'CMAU1246605'}},
 {'_id': '5dc31c17cdd9fc3e8b883593',
  'metadata': {'dataescan

In [62]:
# lista = [['MSCU6656780', '']]
lista = [*lista_lce, *lista_null]
conteineres_ids = {linha['metadata']['numeroinformado']: linha['_id'] for linha in lista}
conteineres = list(conteineres_ids.keys())

print(conteineres_ids)


{'MNBU3980129': '5dc31beacdd9fc3e8b8828a0', 'MRKU9909998': '5dc31bebcdd9fc3e8b8829ad', 'TCLU2385790': '5dc31bf5cdd9fc3e8b882c1c', 'MSKU2319040': '5dc31c0dcdd9fc3e8b88330f', 'MNBU3841632': '5dc31c0dcdd9fc3e8b883373', 'MNBU0401157': '5dc31c0dcdd9fc3e8b88336f', 'CMAU1246605': '5dc31c2ecdd9fc3e8b883b55', 'SUDU6106982': '5dc31c17cdd9fc3e8b883593', 'MNBU0215996': '5dc31c23cdd9fc3e8b8838f5', 'MNBU9051316': '5dc31c2dcdd9fc3e8b883ae9', 'MNBU3957900': '5dc31c2dcdd9fc3e8b883ae5', 'PONU0079802': '5dc31c2dcdd9fc3e8b883a71', 'TEMU3170006': '5dc31bc9cdd9fc3e8b881fd5', 'TGCU0230609': '5dc31bcacdd9fc3e8b8820ab', 'MRKU4476621': '5dc31bc8cdd9fc3e8b881f89', 'CCLU3502299': '5dc31bc7cdd9fc3e8b881e9d', 'TLLU4250703': '5dc31be9cdd9fc3e8b882794', 'VTGU1262054': '5dc31be9cdd9fc3e8b8827a0', 'DFSU7584893': '5dc31bcacdd9fc3e8b882077', 'FCIU2270601': '5dc31bcacdd9fc3e8b88207b', 'MSKU8466583': '5dc31bcacdd9fc3e8b882113', 'CAXU9276502': '5dc31bcacdd9fc3e8b88206b', 'SUDU8530259': '5dc31bcacdd9fc3e8b88212f', 'MSCU92900

## Conectar na API do Pucomex/suiterfb para encontrar e baixar DUE 

In [63]:
# %%time
from selenium import webdriver

GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
driver = webdriver.Chrome(GECKO_PATH)
raspa_due.auth_suite_rfb(driver)
try:
    %time conteineres_listadue = raspa_due.get_dues_pos_acd(driver, conteineres)
    # print(conteineres_listadue)
    %time due_detalhe = raspa_due.detalha_dues(driver, conteineres_listadue)
    # print(conteineres_due)
finally:
    driver.close()

    

Wall time: 22min 18s
Wall time: 8min 20s


## Conectar na API do virasana para fazer UPLOAD do arquivo JSON resultante

O arquivo seguirá o formato [{_id: {Documento DUE}}]

In [52]:
print(len(conteineres_listadue))
print(len(due_detalhe))

2039
341


In [53]:
for conteiner, dues in list(conteineres_listadue.items())[:40]:
    if dues is not None and len(dues) > 0:
        print(conteiner, dues)
        
for due, conteudo in list(due_detalhe.items())[:10]:
    print(due, json.dumps(conteudo)[:30])

MNBU0098380 ['19BR0014603240']
MRKU7981144 ['19BR0014855010']
MNBU3245947 ['19BR0014441640']
TRLU1713071 ['19BR0014636874']
HASU4371557 ['19BR0014861754']
MRKU4336738 ['19BR0014861754']
CAIU4903402 ['19BR0014861754']
MSKU0988909 ['19BR0014861754']
MRKU4513031 ['19BR0014861754']
MNBU0310458 ['19BR0014531258']
HASU5128433 ['19BR0015225185']
MSKU1042264 ['19BR0014861754']
MRKU4856447 ['19BR0014973412']
MSKU0372317 ['19BR0014973412']
HLXU3477447 ['19BR0014438665']
MSKU1769839 ['19BR0014973412']
MRKU4210143 ['19BR0014973412']
MSKU9260684 ['19BR0014973412']
TTNU8217496 ['19BR0014752904']
TRIU8044800 ['19BR0014752912']
MNBU9017029 ['19BR0014108703']
GESU5840127 ['19BR0015023191']
MIEU3061132 ['19BR0015023191']
MRSU3601336 ['19BR0014892188']
MRKU5633094 ['19BR0015023191']
MRKU2397288 ['19BR0015023191']
SUDU6218180 ['19BR0015008869']
SUDU6083280 ['19BR0014855800']
SUDU7501620 ['19BR0015146277']
MRKU6749638 ['19BR0015133701']
GCNU1335402 ['19BR0015175471']
FCIU6088112 ['19BR0015125334']
TGHU0325

In [54]:
with open('dues.txt', 'w') as out:
    json.dump(due_detalhe, out)

In [55]:
with open('dues.txt', 'r') as due_in:
    dues_ = json.load(due_in)

In [38]:
dues_['19BR0015082724']

{'canal': 'VERDE',
 'canalBD': 1,
 'chaveAcesso': '19SBA025569377',
 'dataProcessamentoTa': '2019-11-04T06:44:58.000+0000',
 'exportadorEstrangeiro': False,
 'declarantePJBD': '01996609000125',
 'descricaoResultadoProcessarTA': 'Pendência não impeditiva do embarque',
 'despachoEmRecintoAlfandegado': True,
 'embarqueEmRecintoAlfandegado': True,
 'despachoEmRecintoDomiciliar': False,
 'formaExportacao': 'Por conta própria',
 'formaExportacaoBD': 1,
 'id': 2587996,
 'indicadorDAT': False,
 'indicadorExigencia': False,
 'indicadorImpedeEmbarqueTA': 'N',
 'indicadorInspecao': 'N',
 'indicadorOEA': False,
 'informacoesComplementares': 'JOÃO FLAVIO DE CARVALHO - CPF: 133.732.358-63 - REG: 8D.03.379\nREGINALDO JOÃO DE CARVALHO - CPF:247.658.238-73\nFATURA:279/2019',
 'usuarioConfaz': False,
 'consultaConfaz': False,
 'listaExigencias': [],
 'listaHistorico': [{'dataHoraEvento': '01/11/2019 - 14:26:04',
   'evento': 'Registro',
   'responsavel': '13373235863',
   'tipoEvento': 'REGISTRADA'},
  

In [42]:
def monta_due_ajna(due):
    def get_dados_recinto(recinto_dict):
        result = {}
        try:
            result['codigo'] = recinto_dict.get('codigo')
            depositario = recinto_dict.get('depositario')
            if depositario:
                result['depositario'] = depositario.get('depositario')
                result['nome'] = depositario.get('nome')
                result['descricao'] = depositario.get('descricao')
                unidade = depositario.get('unidadeLocalRFB')
                if unidade:
                    unidadeLocalRFB = depositario.get('codigo')
        except AttributeError:
            return None

        return result 
    
    keys = ['canal', 'chaveAcesso', 'dataProcessamentoTa', 'descricaoTipoItemDue',
            'exportadorEstrangeiro', 'formaExportacao', 'indicadorOEA', 'informacoesComplementares']
    pacote = {}
    for key in keys:
        item = due.get(key)
        if item is not None:
            pacote[key] = item

    declarante = due.get('niDeclarante')
    if declarante:
        pacote['Declarante'] = declarante.get('numero')
        pacote['Nome Declarante'] = declarante.get('nome')

    destino = due.get('paisImportador')
    if destino:
        pacote['PaisImportador'] = destino.get('nome')

    lista_items = due.get('listaInfoItemDue')
    itensDue = []
    for item in lista_items:
        itemDue = {}
        itemDue['descricaoMercadoria'] = item.get('descricaoMercadoria')
        itemDue['exportadorEstrangeiro'] = item.get('exportadorEstrangeiro')
        ncm = item.get('ncm')
        if ncm:
            itemDue['ncm'] = ncm.get('codigo')
        exportador = item.get('niExportador')
        if exportador:
            itemDue['Exportador'] = exportador.get('numero')
            itemDue['NomeExportador'] = exportador.get('nome')
        itensDue.append(itemDue)
    pacote['itens'] = itensDue
    
    for recinto_tipo in ['recintoAduaneiroDespacho',  'recintoAduaneiroEmbarque']:
        pacote[recinto_tipo] = get_dados_recinto(due.get(recinto_tipo))
    ruc = due.get('ruc')
    if ruc:
        pacote['ruc'] = ruc.get('numero')
    pacote['listaHistorico'] = due.get('listaHistorico')

    return pacote

    


In [43]:
pacote = monta_due_ajna(dues_['19BR0015082724'])
pacote

{'canal': 'VERDE',
 'chaveAcesso': '19SBA025569377',
 'dataProcessamentoTa': '2019-11-04T06:44:58.000+0000',
 'descricaoTipoItemDue': ' Nota Fiscal Eletrônica (NF-e)',
 'exportadorEstrangeiro': False,
 'formaExportacao': 'Por conta própria',
 'indicadorOEA': False,
 'informacoesComplementares': 'JOÃO FLAVIO DE CARVALHO - CPF: 133.732.358-63 - REG: 8D.03.379\nREGINALDO JOÃO DE CARVALHO - CPF:247.658.238-73\nFATURA:279/2019',
 'Declarante': '01996609000125',
 'Nome Declarante': 'SUPRACITRUS COMERCIAL LTDA.                                                                                                                           ',
 'PaisImportador': 'PAISES BAIXOS (HOLANDA)',
 'itens': [{'descricaoMercadoria': 'LIMAO TAHITI IN-NATURA,ACONDICIONADO EM EMBALAGEM GRAFICA PERSONALIZADA(PESO LIQUIDO 4,5KG DE LIMAO TAHITI IN- NATURA)',
   'exportadorEstrangeiro': False,
   'ncm': '8055000',
   'Exportador': '01996609000125',
   'NomeExportador': 'SUPRACITRUS COMERCIAL LTDA.                      

In [64]:
pacote_carregamento = {}
for conteiner, numeros_dues in conteineres_listadue.items():
    _id = conteineres_ids[conteiner]
    lista_dues = []
    for due in numeros_dues:
        if due is not None:
            pacote = monta_due_ajna(due_detalhe[due])
            lista_dues.append({'numero': due, **pacote})
    if numeros_dues and len(numeros_dues) > 0:
        pacote_carregamento[_id] = lista_dues
    

In [67]:
r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento, verify=False)
print(r.status_code)
print(r.text)


c:\users\25052288840\pycharmprojects\ajna_docs\virasana\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



201
{
  "status": "DUEs inseridas/atualizadas"
}

